<h1 style="background:black; color:white;"> <b>AMAD</b>: AKKODIS MDAO Aircraft Demonstrator</h1>

In [ ]:
import ipywidgets as widgets
import plotly.graph_objects as go
import pyvista
import numpy
from plotly.subplots import make_subplots
import plotly.express as px
from cosapp.drivers import Optimizer, NonLinearSolver
from amad.disciplines.design.systems import GenerateAeroGeom
from amad.disciplines.aerodynamics.tools.flightVehicleViewer import FlightVehicleViewer
from amad.demo.demo_aircraft_resources import CalculateAircraft
from amad.optimization.resources.set_parameters import single_aisle_concept, advanced_ac_concept
from amad.tools.atmosBADA import AtmosphereAMAD

# set up viz aircraft
opt = single_aisle_concept(GenerateAeroGeom('opt'))
opt.run_once()
airplane_geom = opt.asb_aircraft_geometry_out

# run the performance calc for the baseline aircraft
tolerance=1e-2
calc_perf_init = CalculateAircraft('calc_perf', airplane_geom=airplane_geom)
calc_perf = single_aisle_concept(calc_perf_init)
calc_perf.add_driver(NonLinearSolver('nls', tol=tolerance))
# calc_perf.cruise_fuel.m_gross.mass = 48876.  # initial mass
calc_perf.m_fuel_climb = 2300.
calc_perf.m_fuel_descent = 300.
calc_perf.m_fuel_taxi = 500.
calc_perf.z_altitude = 10058.4  # cruise altitude
calc_perf.mach_current = .8  # cruise mach
calc_perf.thrust_eng = 120000.
calc_perf.x_range = 5000 * 1000.  # 5000km

calc_perf.run_drivers()

## Capture Reference Values ##############################################################
##########################################################################################

# Performance / cost
fuel_cost_kg = 1.119
n_pax = calc_perf.n_pax
v_tas = calc_perf.cruise_fuel.v_tas
ref_fuel = calc_perf.mass.fuel.total.mass

ref_spec_fuel = ref_fuel / (calc_perf.x_range / 1000)
ref_mtow = calc_perf.m_mzf
ref_trip_time = (calc_perf.x_range / v_tas) / 60
ref_fuel_cost = ref_fuel * fuel_cost_kg
ref_pax_cost = 100 * (ref_fuel_cost / n_pax) / (calc_perf.x_range / 1000)

# lift/drag interpolators
cl_int_ref = calc_perf.cruise_fuel.equi.cl_int
cd_int_ref = calc_perf.cruise_fuel.equi.cd_int
cl_list = []
cd_list = []

# mass
# create breakdown dictionary

mass_source = calc_perf.mass

mass_components = [
    'Aircraft',
    'Manufacturer Empty Weight',
    'Structure',
    'Fuel',
    'Powerplant',
    'Systems',
    'Payload',
    'Completion',
    'Wing',
    'Tail',
    'Fuselage',
    'Gear',
    'Nacelles',
    'Climb Fuel',
    'Cruise Fuel',
    'Descent Fuel',
    'Taxi Fuel',
    'Fuel System',
    'Flight Controls',
    'APU',
    'Instruments',
    'Hydraulics',
    'Electrical',
    'Avionics',
    'Air Conditioning',
    'Anti-Ice System',
    'Cabin Furnishings',
]

mass_parents = [
    '',
    'Aircraft',
    'Manufacturer Empty Weight',
    'Aircraft',
    'Manufacturer Empty Weight',
    'Manufacturer Empty Weight',
    'Aircraft',
    'Manufacturer Empty Weight',
    'Structure',
    'Structure',
    'Structure',
    'Structure',
    'Structure',
    'Fuel',
    'Fuel',
    'Fuel',
    'Fuel',
    'Systems',
    'Systems',
    'Systems',
    'Systems',
    'Systems',
    'Systems',
    'Systems',
    'Systems',
    'Systems',
    'Systems',
]

mass_values = [
    mass_source.total_mass,
    mass_source.m_zfw - mass_source.payload_mass,
    mass_source.structure_mass,
    mass_source.fuel.total.mass,
    mass_source.powerplant_mass,
    mass_source.systems_mass,
    mass_source.payload_mass,
    mass_source.completion_mass,
    mass_source.wing.total.mass,
    mass_source.tail.total.mass,
    mass_source.fuse.total.mass,
    mass_source.gear.total.mass,
    mass_source.nacelle.total.mass,
    mass_source.m_fuel_climb,
    mass_source.m_fuel_cruise,
    mass_source.m_fuel_descent,
    mass_source.m_fuel_taxi,
    mass_source.sys.m_fuel_system,
    mass_source.sys.m_surface_controls,
    mass_source.sys.m_apu,
    mass_source.sys.m_instruments,
    mass_source.sys.m_hydraulics,
    mass_source.sys.m_electrical,
    mass_source.sys.m_avionics,
    mass_source.sys.m_air_conditioning,
    mass_source.sys.m_anti_ice,
    mass_source.sys.m_furnishings,
]

mass_data = dict(
    component=mass_components,
    parent=mass_parents,
    mass=mass_values
)

mass_data_reference = [
    mass_source.total_mass,
    mass_source.fuel.total.mass,
    mass_source.structure_mass,
    mass_source.payload_mass,
    mass_source.systems_mass,
]

mass_data_updated = []

## Helper Functions ######################################################################
##########################################################################################
def reset_slider_values():
    i = 1 if ac_picker.value == 'New Concept' else 0
    design_sliders[0].value = [189, 189][i]
    design_sliders[1].value = [23.5, -25][i]
    design_sliders[2].value = [8.57, 10][i]
    design_sliders[3].value = [.252, .129][i]
    design_sliders[4].value = [34.32, 34.32][i]
    perf_sliders[0].value = [33000, 33000][i]
    perf_sliders[1].value = [5000, 5000][i]
    perf_sliders[2].value = [.8, .8][i]
    perf_sliders[3].value = [100, 100][i]

def update_aircraft_parameters(ac_models: list):
    # update model values
    inner_sweep = (31.5 / 23.5) * design_sliders[1].value

    # Load correct model
    for model in ac_models:

        if ac_picker.value == 'New Concept':
            model = advanced_ac_concept(model)
            model.delta_wing_sweep = [design_sliders[1].value]
            model.r_wing_taper = [design_sliders[3].value]
            model.x_wing_span = [design_sliders[4].value]
        else:
            model = single_aisle_concept(model)
            model.delta_wing_sweep = [inner_sweep, design_sliders[1].value]
            model.r_wing_taper = [.631, design_sliders[3].value]
            model.x_wing_span = [model.x_wing_span[0], design_sliders[4].value]

        model.n_pax = design_sliders[0].value
        model.chord_wing_root = design_sliders[2].value

        try:
            model.z_altitude = perf_sliders[0].value * .3048
            model.x_range = perf_sliders[1].value * 1000
            model.mach_current = perf_sliders[2].value
            model.f_eng_efficiency = perf_sliders[3].value / 100
        except AttributeError:
            continue

def update_aircraft_visu(ac_model, vis_index: int):
    ac_model.run_drivers()
    air_geom_data = FlightVehicleViewer(flight_vehicle_geom=ac_model.asb_aircraft_geometry_out, plot=False, backend='plotly')

    go_airplane_list[vis_index].data[0].update(
        i=air_geom_data.data[0]['i'],
        j=air_geom_data.data[0]['j'],
        k=air_geom_data.data[0]['k'],
        x=air_geom_data.data[0]['x'],
        y=air_geom_data.data[0]['y'],
        z=air_geom_data.data[0]['z'],
    )

    if ac_picker.value == 'New Concept':
        go_airplane_list[vis_index]['layout']['scene']['zaxis']['range'] = (-2.259540468372135, 28.511371449951946)
    else:
        go_airplane_list[vis_index]['layout']['scene']['zaxis']['range'] =(-2.259540468372135, 8.511371449951946)


def update_charts():
    # Aero charts
    cl_int_ref = calc_perf.cruise_fuel.equi.cl_int
    cd_int_ref = calc_perf.cruise_fuel.equi.cd_int
    aero_graphs[0].data[1].x = cd_int_ref(alpha_list)
    aero_graphs[0].data[1].y = cl_int_ref(alpha_list)
    aero_graphs[1].data[1].y = cl_int_ref(alpha_list)

    # mass props
    mass_graphs.data[1]['y'] = [
        calc_perf.mass.total_mass,
        calc_perf.mass.fuel.total.mass,
        calc_perf.mass.structure_mass,
        calc_perf.mass.payload_mass,
        calc_perf.mass.systems_mass,
    ]

    # gauges
    upd_spec_fuel = calc_perf.mass.fuel.total.mass / (calc_perf.x_range/1000)
    upd_fuel_cost = calc_perf.mass.fuel.total.mass * fuel_cost_kg
    upd_pax_cost = 100 * (upd_fuel_cost / n_pax) / (calc_perf.x_range / 1000)

    gauge_graphs[0].data[0].value = upd_spec_fuel
    gauge_graphs[1].data[0].value = upd_fuel_cost
    gauge_graphs[2].data[0].value = upd_pax_cost

    gauge_graphs[0].layout.annotations[0].text = f'{1 - (ref_spec_fuel / upd_spec_fuel):.1%}'
    gauge_graphs[1].layout.annotations[0].text = f'{1 - (ref_fuel_cost / upd_fuel_cost):.1%}'
    gauge_graphs[2].layout.annotations[0].text = f'{1 - (ref_pax_cost / upd_pax_cost):.1%}'

def update_button_colors(button_index):
    ec = int(calc_perf.drivers['nls'].error_code)
    if ec > 0:
        control_buttons[button_index].style.button_color = 'red'
    else:
        control_buttons[button_index].style.button_color = 'lightgreen'

def ac_picker_change(b):
    reset_slider_values()
    update_aircraft_parameters([opt, calc_perf])
    update_aircraft_visu(opt, 0)
    update_aircraft_visu(opt, 1)

def reset_button_press(b):
    reset_slider_values()

def update_viz_button_press(b):
    update_aircraft_parameters([opt])
    update_aircraft_visu(opt, 1)

def run_perf_button_press(b):
    control_buttons[2].style.button_color = 'orange'
    update_aircraft_parameters([opt, calc_perf])
    update_aircraft_visu(opt, 1)
    calc_perf.drivers.clear()
    calc_perf.add_driver(NonLinearSolver('nls', method='POWELL', tol=tolerance))
    calc_perf.run_drivers()
    update_button_colors(2)
    update_charts()

def run_mdao_button_press(b):
    control_buttons[3].style.button_color = 'orange'
    # Update parameters to cached values found during previous optimization
    design_sliders[1].value = 25.6  # sweep
    design_sliders[4].value = 38.9  # span
    design_sliders[2].value = 9.8  # chord
    update_aircraft_parameters([opt, calc_perf])
    update_aircraft_visu(opt, 1)
    calc_perf.drivers.clear()
    calc_perf.add_driver(NonLinearSolver('nls', method='POWELL', tol=tolerance))
    calc_perf.run_drivers()
    update_button_colors(3)
    update_charts()

## Control Widgets #######################################################################
##########################################################################################

## Setup #################################################################################
slider_width_int = 400
slider_width = f'{slider_width_int}px'

slider_height_int = 65
slider_height = f'{slider_height_int}px'

desc_style = {'description_width': 'initial'}

## HTML Titles ###########################################################################
title_design = widgets.HTML(value="<h2>Design Variables</h2>")
title_performance = widgets.HTML(value="<h2>Performance Variables</h2>")
title_compute = widgets.HTML(value="<h2>Compute</h2>")

## Dropdown aircraft picker ##############################################################
ac_picker = widgets.Dropdown(
    options=['Standard Concept', 'New Concept'],
    value='Standard Concept',
    description='Aircraft:',
    disabled=False,
    layout=widgets.Layout(width=slider_width),
)
# ac_picker.observe(ac_picker_change)

## Buttons ###############################################################################
but_w = f'{slider_width_int / 2}px'
but_h = '50px'

def gen_button(parameters: dict):
    button = widgets.Button(
        description=parameters['desc'],
        disabled=False,
        tooltip=parameters['ttip'],
        icon=parameters['icon'], # (FontAwesome names without the `fa-` prefix)
        layout=widgets.Layout(width=but_w, height=but_h),
    )
    return button

button_data = {
    'reset': {'desc': 'Reset Values', 'ttip': 'Reset', 'icon': 'arrow-up'},
    'update': {'desc': 'Update 3D models', 'ttip': '3D Models update', 'icon': 'cube'},
    'run_perf': {'desc': 'Run Analysis', 'ttip': 'Launch', 'icon': 'plane'},
    'run_mdao': {'desc': 'Run Optimization', 'ttip': 'Launch', 'icon': 'rocket'},
}

control_buttons = [gen_button(button_data[p]) for p in button_data]
control_buttons[2].style.button_color = 'lightgreen'
control_buttons[3].style.button_color = 'lightgreen'
# reset_button_perf.on_click(reset_slider_values)

control_buttons[0].on_click(reset_button_press)
control_buttons[1].on_click(update_viz_button_press)
control_buttons[2].on_click(run_perf_button_press)
control_buttons[3].on_click(run_mdao_button_press)

button_line1 = widgets.HBox([control_buttons[0], control_buttons[1]])
button_line2 = widgets.HBox([control_buttons[2], control_buttons[3]])
button_box = widgets.VBox([button_line1, button_line2])

## Control Sliders #######################################################################

def gen_slider(parameters: dict):
    generated_slider = widgets.FloatSlider(
        value=parameters['value'],
        min=parameters['min'],
        max=parameters['max'],
        step=parameters['step'],
        description=parameters['desc'],
        style=desc_style,
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format=parameters['n_form'],
        layout=widgets.Layout(width=slider_width, height=slider_height)
    )
    return generated_slider

design_slider_data = {
    'n_passengers': {'value': calc_perf.n_pax, 'min': 150, 'max': 250, 'step': 1, 'desc': 'Number of Passengers', 'n_form': '.0f'},
    'wingsweep': {'value': calc_perf.delta_wing_sweep[1], 'min': -40, 'max': 40, 'step': .1, 'desc': 'Wing sweep (deg)', 'n_form': '.1f'},
    'chord_root': {'value': calc_perf.chord_wing_root, 'min': 5, 'max': 10, 'step': .5, 'desc': 'Wing root chord (m)', 'n_form': '.1f'},
    'taper_rat': {'value': calc_perf.r_wing_taper[1], 'min': .1, 'max': .5, 'step': .01, 'desc': 'Wing taper ratio', 'n_form': '.3f'},
    'span_len': {'value': calc_perf.x_wing_span[1], 'min': 10, 'max': 40, 'step': .5, 'desc': 'Wing span', 'n_form': '.1f'},
}

perf_slider_data = {
    'cruise_alt': {'value': calc_perf.z_altitude / .3048, 'min': 25000, 'max': 40000, 'step': 500, 'desc': 'Cruise altitude (ft)', 'n_form': '.0f'},
    'flight_range': {'value': calc_perf.x_range / 1000, 'min': 2500, 'max': 10000, 'step': 500, 'desc': 'Mission range (km)', 'n_form': '.0f'},
    'cruise_mach': {'value': calc_perf.mach_current, 'min': .7, 'max': .9, 'step': .01, 'desc': 'Cruise mach', 'n_form': '.2f'},
    'eng_efficiency': {'value': calc_perf.f_eng_efficiency * 100, 'min': 50, 'max': 150, 'step': 2, 'desc': 'Engine fuel conso fact (%)', 'n_form': '.0f'},
}

design_sliders = [gen_slider(design_slider_data[p]) for p in design_slider_data]
perf_sliders = [gen_slider(perf_slider_data[p]) for p in perf_slider_data]

input_control_list = [title_design] + [ac_picker] + design_sliders + [title_performance] + perf_sliders + [title_compute] + [button_box]
input_controls = widgets.VBox(input_control_list)

## Charts and Graphs #####################################################################
##########################################################################################

## General Settings ######################################################################
graph_scale = 340
graph_margins = 5
plt_bgcol = 'rgba(0,0,0,0)'
pap_bgcol = '#cfe2f3'
color_reference = '#08456e'
color_updated = '#FFB81C'

## Mass Props ############################################################################

mass_data_labels = ['Total', 'Fuel', 'Structures', 'Payload', 'Systems']

def generate_mass_graphs():
    mass_barchart = go.FigureWidget()

    mass_barchart.add_trace(
        go.Histogram(
            histfunc="sum",
            x=mass_data_labels,
            y=mass_data_reference,
            name="Reference AC Mass (kg)",
            marker_color=color_reference,
        )
    )

    mass_barchart.add_trace(
        go.Histogram(
            histfunc="sum",
            x=mass_data_labels,
            y=mass_data_updated,
            name="Updated Ac Mass (kg)",
            marker_color=color_updated,
        )
    )

    mass_barchart.update_layout(
            template='plotly_white',
            width=graph_scale,
            height=graph_scale,
            paper_bgcolor = pap_bgcol,
            plot_bgcolor = plt_bgcol,
            margin=dict(l=graph_margins, r=graph_margins, t=graph_margins, b=graph_margins),
            legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="right",
                x=0.99,
            )
    )

    mass_barchart.update_xaxes(title_text="Component")
    mass_barchart.update_yaxes(title_text="Mass (kg)")

    return mass_barchart

mass_graphs = generate_mass_graphs()

# mass_fig_sunburst = go.FigureWidget()
# mass_fig_sunburst.add_traces(go.Sunburst(
#     mass_data,
#     names='component',
#     parents='parent',
#     values='mass',
#     color='mass',
#     color_continuous_scale='YlGn',
#     height=1000,
#     title='Aircraft Weight Breakdown'
# ))


# mass_fig_treemap = px.treemap(
#     mass_data,
#     names='component',
#     parents='parent',
#     values='mass',
#     color='mass',
#     color_continuous_scale='YlGn',
#     height=1000,
#     title='Aircraft Weight Breakdown'
# )

# mass_widgets = widgets.VBox([mass_fig_sunburst, mass_fig_treemap])


## Aerodynamic Charts ####################################################################
alpha_list = numpy.linspace(-10., 10., 1000)
cl_list_ref = cl_int_ref(alpha_list)
cd_list_ref = cd_int_ref(alpha_list)
cl_cd = [cl / cd for cl, cd in zip(cl_list, cd_list)]

def generate_aero_graphs():
    cl_cd = go.FigureWidget()
    cl_cd_alpha = go.FigureWidget()

    cl_cd.add_trace(
        go.Scatter(
            x=cd_list_ref,
            y=cl_list_ref,
            name="Reference CL vs CD",
            marker=dict(color=color_reference),
        )
    )
    cl_cd.add_trace(
        go.Scatter(
            x=cd_list,
            y=cl_list,
            name="Updated CL vs CD",
            marker=dict(color=color_updated),
        )
    )
    cl_cd.update_xaxes(title_text="CD", range=[0, .12])
    cl_cd.update_yaxes(title_text="CL", range=[-1, 1.5])

    cl_cd_alpha.add_trace(
        go.Scatter(
            x=alpha_list,
            y=cl_list_ref,
            name="Reference CL vs Alpha",
            marker=dict(color=color_reference),
        )
    )

    cl_cd_alpha.add_trace(
        go.Scatter(
            x=alpha_list,
            y=cl_list,
            name="Updated CL vs Alpha",
            marker=dict(color=color_updated),
        )
    )

    cl_cd_alpha.update_xaxes(title_text="Alpha")
    cl_cd_alpha.update_yaxes(title_text="CL")

    for graph in [cl_cd, cl_cd_alpha]:
        graph.update_traces(showlegend=True)
        graph.update_layout(
            template='plotly_white',
            width=graph_scale,
            height=graph_scale,
            paper_bgcolor = pap_bgcol,
            plot_bgcolor = plt_bgcol,
            margin=dict(l=graph_margins, r=graph_margins, t=graph_margins, b=graph_margins),
            legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="left",
                x=0.01
            )
        )

    return (cl_cd, cl_cd_alpha)


## Gauges ################################################################################

def gen_gauges(parameters: dict):
    gauge = go.FigureWidget()
    gauge.add_trace(go.Indicator(
        mode = "gauge+number+delta",
        value = parameters['value'],
        delta = {'reference': parameters['delta'], 'increasing': {'color': "red"}, 'decreasing': {'color': "green"}},
        gauge = {
            'axis': {'range': parameters['ax_range']},
            'bar': {'color': color_updated},
            'threshold' : {'line': {'color': color_reference, 'width': 4}, 'thickness': 0.75, 'value': parameters['delta']}
            },
        title = {'text': parameters['title']})
    )

    gauge.add_annotation(x=.5, y=.55, text="", showarrow=False, font=dict(size=18))
    gauge.update_layout(
            template='plotly_white',
            width=graph_scale,
            height=graph_scale,
            paper_bgcolor = pap_bgcol,
            plot_bgcolor = "#FFFFFF",
            margin=dict(l=30, r=30, t=30, b=graph_margins),
            # legend=dict(
            #     yanchor="top",
            #     y=0.99,
            #     xanchor="left",
            #     x=0.01
            # )
        )

    return gauge

upd_spec_fuel = None
upd_fuel_cost = None
upd_pax_cost = None

gen_gauge_data = {
    'f_conso': {'value': upd_spec_fuel, 'delta': ref_spec_fuel, 'ax_range': [None, 10], 'title': 'Fuel conso / km (kg)'},
    'f_cost_f': {'value': upd_fuel_cost, 'delta': ref_fuel_cost, 'ax_range': [None, 40000], 'title': 'Fuel cost / flight (US$)'},
    'f_cost_p': {'value': upd_pax_cost, 'delta': ref_pax_cost, 'ax_range': [None, 5], 'title': 'Fuel cost / 100 pax km (US$)'},
}

gauge_graphs = [gen_gauges(gen_gauge_data[p]) for p in gen_gauge_data]

## 3D Plotly Airplane ####################################################################
##########################################################################################

air_geom_dimension_x = 600
air_geom_dimension_y = 525
air_geom_bg_col = '#e7f2fb'
# axis_settings = {'showgrid': False, 'showbackground': False, 'showline': False, 'showticklabels': False}
axis_settings = {'showgrid': False, 'showbackground': False, 'showline': False, 'showticklabels': False}

air_geom_data_reference = FlightVehicleViewer(flight_vehicle_geom=opt.asb_aircraft_geometry_out, plot=False, backend='plotly')
air_geom_data_reference.data[0].update(showscale=False, showlegend=False, colorscale=[[0, color_reference], [1, color_reference]], lighting=dict(ambient=0.4, diffuse=0.5, roughness=0.6, specular=2., fresnel=0.2))
air_geom_data_reference.data[1].update(x=[], y=[], z=[])

air_geom_data_updated = FlightVehicleViewer(flight_vehicle_geom=opt.asb_aircraft_geometry_out, plot=False, backend='plotly')
air_geom_data_updated.data[0].update(showscale=False, showlegend=False, colorscale=[[0, color_updated], [1, color_updated]], lighting=dict(ambient=0.4, diffuse=0.5, roughness=0.6, specular=2., fresnel=0.2))
air_geom_data_updated.data[1].update(x=[], y=[], z=[])

go_airplane_reference = go.FigureWidget(air_geom_data_reference)
go_airplane_reference.layout = go.Layout(
    width=air_geom_dimension_x,
    height=air_geom_dimension_y,
    template='plotly_white',
    paper_bgcolor=air_geom_bg_col,
    plot_bgcolor='#FFFFFF',
    margin=dict(l=graph_margins, r=graph_margins, t=graph_margins, b=graph_margins),
    scene=dict(xaxis=axis_settings, yaxis=axis_settings, zaxis=axis_settings)
)

go_airplane_updated = go.FigureWidget(air_geom_data_updated)
go_airplane_updated.layout = go.Layout(
    width=air_geom_dimension_x,
    height=air_geom_dimension_y,
    template='plotly_white',
    paper_bgcolor=air_geom_bg_col,
    plot_bgcolor='#FFFFFF',
    margin=dict(l=graph_margins, r=graph_margins, t=graph_margins, b=graph_margins),
    scene=dict(xaxis=axis_settings, yaxis=axis_settings, zaxis=axis_settings)
)

go_airplane_list = [go_airplane_reference, go_airplane_updated]

go_airplane_vbox = widgets.VBox(go_airplane_list)


## Final Layout ##########################################################################
##########################################################################################

aero_graphs = [*generate_aero_graphs()]
output_col1 = widgets.VBox([aero_graphs[0], gauge_graphs[0], gauge_graphs[1]])
output_col2 = widgets.VBox([aero_graphs[1], mass_graphs, gauge_graphs[2]])
output_charts = widgets.HBox([output_col1, output_col2])


## Display Widgets #######################################################################
##########################################################################################

dashboard_contents = widgets.HBox([input_controls, go_airplane_vbox, output_charts])
display(dashboard_contents)

In [ ]:

mass_source = calc_perf.mass

mass_components = [
    'Aircraft',
    'Manufacturer Empty Weight',
    'Structure',
    'Fuel',
    'Powerplant',
    'Systems',
    'Payload',
    'Completion',
    'Wing',
    'Tail',
    'Fuselage',
    'Gear',
    'Nacelles',
    'Climb Fuel',
    'Cruise Fuel',
    'Descent Fuel',
    'Taxi Fuel',
    'Fuel System',
    'Flight Controls',
    'APU',
    'Instruments',
    'Hydraulics',
    'Electrical',
    'Avionics',
    'Air Conditioning',
    'Anti-Ice System',
    'Cabin Furnishings',
]

mass_parents = [
    '',
    'Aircraft',
    'Manufacturer Empty Weight',
    'Aircraft',
    'Manufacturer Empty Weight',
    'Manufacturer Empty Weight',
    'Aircraft',
    'Manufacturer Empty Weight',
    'Structure',
    'Structure',
    'Structure',
    'Structure',
    'Structure',
    'Fuel',
    'Fuel',
    'Fuel',
    'Fuel',
    'Systems',
    'Systems',
    'Systems',
    'Systems',
    'Systems',
    'Systems',
    'Systems',
    'Systems',
    'Systems',
    'Systems',
]

mass_values = [
    mass_source.total_mass,
    mass_source.m_zfw - mass_source.payload_mass,
    mass_source.structure_mass,
    mass_source.fuel.total.mass,
    mass_source.powerplant_mass,
    mass_source.systems_mass,
    mass_source.payload_mass,
    mass_source.completion_mass,
    mass_source.wing.total.mass,
    mass_source.tail.total.mass,
    mass_source.fuse.total.mass,
    mass_source.gear.total.mass,
    mass_source.nacelle.total.mass,
    mass_source.m_fuel_climb,
    mass_source.m_fuel_cruise,
    mass_source.m_fuel_descent,
    mass_source.m_fuel_taxi,
    mass_source.sys.m_fuel_system,
    mass_source.sys.m_surface_controls,
    mass_source.sys.m_apu,
    mass_source.sys.m_instruments,
    mass_source.sys.m_hydraulics,
    mass_source.sys.m_electrical,
    mass_source.sys.m_avionics,
    mass_source.sys.m_air_conditioning,
    mass_source.sys.m_anti_ice,
    mass_source.sys.m_furnishings,
]

mass_data = dict(
    component=mass_components,
    parent=mass_parents,
    mass=mass_values
)

mass_data_reference = [
    mass_source.total_mass,
    mass_source.fuel.total.mass,
    mass_source.structure_mass,
    mass_source.payload_mass,
    mass_source.systems_mass,
]

fig = px.sunburst(
    mass_data,
    names='component',
    parents='parent',
    values='mass',
    color='mass',
    color_continuous_scale='YlGn',
    height=1000,
    title='Aircraft Weight Breakdown'
)
fig.show()